In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import PowerTransformer, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score


import numpy as np
import pandas as pd
import os
import sys

sys.path.append('../src/')
from dataloader import *
from logging_utils import *


import mlflow
import optuna

# start mlflow server from terminal: $mlflow server mlflow server --host 127.0.0.1 --port 8080
mlflow.set_tracking_uri("http://127.0.0.1:8080")
optuna.logging.set_verbosity(optuna.logging.ERROR)

%reload_ext autoreload

## Load Data

In [2]:
PATH = '../data/'

df = load_train_df(
    PATH,
    decode_dummies=True,
    add_geo_features=True
    )

In [3]:
y = df['Cover_Type']
X = df.drop(['Cover_Type'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
test_model = RandomForestClassifier(n_estimators=100, random_state=42)
test_model.fit(X_train, y_train)

y_pred = test_model.predict(X_test)
print('Accuracy:', accuracy_score(y_test, y_pred))


Accuracy: 0.8673941798941799


In [5]:
from sklearn.model_selection import cross_val_predict

def objective(trial, experiment_id):
    with mlflow.start_run(experiment_id=experiment_id, nested=True):
        # Define hyperparameters
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 50, 100),
            "max_depth": trial.suggest_int("max_depth", 2, 8),
            "bootstrap": trial.suggest_categorical("bootstrap", [True, False]),
        }

        model = RandomForestClassifier(**params)

        # TODO THE FOLLOWING IS NOT VERY SOUND: CV SCORE AND CMATRIX ON DIFF SPLITS
        acc = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
        f1 = cross_val_score(model, X_train, y_train, cv=3, scoring='f1_macro').mean()
        
        metrics = {
            "accuracy": acc,
            "f1_macro": f1,

        }

        y_pred = cross_val_predict(model, X_train, y_train)
        
        # fig = plot_confusion_matrix(y, y_pred)
        # mlflow.log_figure(fig, "confusion_matrix.png")

        mlflow.log_params(params)
        mlflow.log_metrics(metrics)
        # NOTE model is not logged to mlflow: eval if worth and then save it

        return acc


In [7]:
# NOTE: create a new experiment whenever the data changes
# create mlflow experiment
confirm = input("Launch new mlflow experiment? (y/n)")
if confirm == "y":
    experiment_id = get_or_create_experiment("First Optuna Experiment")
    mlflow.set_experiment(experiment_id=experiment_id)

    run_name = input("Enter run name: ")
    with mlflow.start_run(
        experiment_id=experiment_id,
        run_name=run_name
        ):

        study = optuna.create_study(direction="maximize")

        study.optimize(
            lambda trial: objective(trial, experiment_id),
            n_trials=3,
            callbacks=[champion_callback]
            )

        mlflow.log_params(study.best_params)

        best_model = RandomForestClassifier(**study.best_params)
        best_model.fit(X_train, y_train)
        y_pred = best_model.predict(X_test)

        metrics = {
            "accuracy": accuracy_score(y_test, y_pred),
            "f1_macro": f1_score(y_test, y_pred, average='macro'),
        }

        cm = plot_confusion_matrix(y_test, y_pred)

        mlflow.log_figure(cm, "confusion_matrix.png")
        mlflow.log_metrics(metrics)
        mlflow.sklearn.log_model(best_model, "best_model")
        mlflow.set_tag("mlflow.note.content", "This is a test run")

else:
    print("No new experiment created")
    sys.exit(0)


Trial 0 value: 0.6895667989417991
